In [ ]:
# example of a cnn for image classification
import cv2
import tensorflow as tf
from numpy import asarray
from numpy import unique
from numpy import argmax
from matplotlib import pyplot
from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from google.colab import drive
from google.colab.patches import cv2_imshow

In [ ]:
# load dataset
(train_images, train_labels), (test_images, test_labels) = load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
# summarize loaded dataset
print("Train: train_images.shape = %s, train_labels.shape = %s" % (train_images.shape, train_labels.shape))
print("Test: test_images.shape = %s, test_labels.shape = %s" % (test_images.shape, test_labels.shape))

In [ ]:
# plot first few images
pyplot.figure(figsize=(10,10))
for i in range(64):
  # define subplot
  pyplot.subplot(8, 8, i+1)
  pyplot.xticks([])
  pyplot.yticks([])
  pyplot.grid(False)
  # plot raw pixel data
  pyplot.imshow(train_images[i], cmap=pyplot.cm.binary)
  pyplot.xlabel(class_names[train_labels[i]])
# show the figure
pyplot.show()

In [ ]:
# reshape data to have a single channel
train_images = train_images.reshape((train_images.shape[0], train_images.shape[1], train_images.shape[2], 1))
test_images = test_images.reshape((test_images.shape[0], test_images.shape[1], test_images.shape[2], 1))
# determine the shape of the input images
in_shape = train_images.shape[1:]
# determine the number of classes
n_classes = len(unique(train_labels))
print('After reshape, train_images.shape = %s, test_images.shape = %s' %(train_images.shape, test_images.shape))
print('in_shape = %s, n_classes = %s' %(in_shape, n_classes))
# normalize pixel values
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

In [ ]:
# define model
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform', input_shape=in_shape))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))
model.summary()

In [ ]:
# define loss and optimizer
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# fit the model
model.fit(train_images, train_labels, epochs=10, batch_size=128, verbose=1)

In [ ]:
# evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=0)
print('Accuracy: %.3f' % acc)

In [ ]:
# make a prediction
image = test_images[222]
yhat = model.predict(asarray([image]))
print("yhat = ", yhat)
print('Predicted: class = %s' % class_names[argmax(yhat)])

In [ ]:
# actual classification of test image
pyplot.figure()
pyplot.imshow(test_images[222].squeeze())
pyplot.xlabel(class_names[test_labels[222]])
pyplot.show()

In [ ]:
# Read an external image
drive.mount("/content/drive")
img = cv2.imread('/content/drive/My Drive/fashion_example01.jpg', cv2.IMREAD_GRAYSCALE)
cv2_imshow(img)
print("Before preprossing, img.shape = ", img.shape)

In [ ]:
# Resize and negate the image
imgsize = 28
img = cv2.bitwise_not(img)
img = cv2.resize(img, (imgsize, imgsize))
cv2_imshow(img)
pyplot.figure()
pyplot.imshow(img)
pyplot.show()

In [ ]:
# Reshape the image for the CNN fashion model
img = tf.expand_dims(img, -1)
img = tf.divide(img, 255)
img = tf.reshape(img, [1, imgsize, imgsize, 1])
print("After preprossing, img.shape = ", img.shape)

In [ ]:
# Predict about the image
yhat = model.predict(asarray(img))
print("yhat = ", yhat)
print('Predicted: class = %s' % class_names[argmax(yhat)])

In [ ]:
# save the model
model.save("/content/drive/My Drive/CNN_Fashion_Colab_Model01")
print('Model saved as CNN_Fashion_Colab_Model01')